In [ ]:
# GPU information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


# Import Libraries

In [ ]:
# import libraries
from matplotlib import pyplot as plt
import seaborn as sb

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
#import random



import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
#from keras import models
from keras.models import load_model

# Load the Dataset

In [ ]:
#from os.path import join
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)

# change directory to the database
%cd '/content/drive/MyDrive/Colab Notebooks/AML_22-23_SN19017126'

# list items
print('\nCurrent directory: \n')
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/AML_22-23_SN19017126

Current directory: 

A1  B1_CNN.h5			     B2_CNN.h5	   main.ipynb
A2  B1_face_shape_recognition.ipynb  Datasets	   __pycache__
B1  B2				     landmarks.py


In [ ]:
# extracting the eye_color label from cartoon dataset
cartoon_label = pd.read_csv('./Datasets/cartoon_set/labels.csv', sep = '\t')
face_label = cartoon_label.drop(columns = [cartoon_label.columns[0]]).drop(columns = [cartoon_label.columns[1]])
face_label['face_shape'] = face_label['face_shape'].apply(str)
print(face_label)

y_train = face_label
print('Shape of y_train: ', y_train.shape)

     face_shape file_name
0             4     0.png
1             4     1.png
2             3     2.png
3             0     3.png
4             2     4.png
...         ...       ...
9995          2  9995.png
9996          3  9996.png
9997          2  9997.png
9998          2  9998.png
9999          2  9999.png

[10000 rows x 2 columns]
Shape of y_train:  (10000, 2)


In [ ]:
# the link of the image 
cartoon_img = ('./Datasets/cartoon_set/img')

In [ ]:
# create data argumentation
data_argumantation = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255.,
    validation_split = 0.25,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    #width_shift_range=[-100, +100],
    #height_shift_range=[-100, +100],
    brightness_range=[0.5, 1.5],
    #zoom_range=[0.5, 1.5]
)

In [ ]:
# Generata batches of image dataset and labels for training
print("Training Dataset: ")
train_generator = data_argumantation.flow_from_dataframe(
    
        dataframe = y_train, 
        directory = cartoon_img,  #path to read images
        x_col = "file_name", 
        y_col = "face_shape",
        class_mode = 'categorical',  #must include the class of each image
        target_size = (128, 128),  #(32,32),
        batch_size = 128, 
        subset = 'training') 
   
#  Generata batches of image dataset and labels for validation 
print("\n")
print("Validation Dataset: ")
validation_generator = data_argumantation.flow_from_dataframe(
        dataframe = y_train, 
        directory = cartoon_img,  #path to read images
        x_col = "file_name", 
        y_col = "face_shape",
        class_mode = 'categorical',  #must include the class of each image
        target_size = (128, 128),  #(32,32),
        batch_size = 128, 
        subset = 'validation')

Training Dataset: 
Found 7500 validated image filenames belonging to 5 classes.


Validation Dataset: 
Found 2500 validated image filenames belonging to 5 classes.


# CNN

In [ ]:
# reset the model
keras.backend.clear_session()

# starting point 
cnn_model= Sequential()

# Block 1
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=(128,128,3)))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Block 2
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Block 3
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Block 4
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Add a dropout layer with a rate of 0.5
cnn_model.add(Dropout(0.5))

# Flatten the output of the convolutional layers into a single vector
cnn_model.add(Flatten())

# Add a fully connected layer with 128 units and a ReLU activation function
cnn_model.add(Dense(128, activation='relu'))

# Add a final fully connected layer with a softmax activation function for classification
num_classes = 5
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model with a categorical cross-entropy loss function and Adam optimization
cnn_model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer='adam', 
                  metrics=['accuracy'])


In [ ]:
# Print the summary of the model and the no. of trainable/non-trainable parameters
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 64)       0

In [ ]:
training_history = cnn_model.fit(
                                train_generator,
                                epochs=20,
                                steps_per_epoch=train_generator.samples // train_generator.batch_size, 
                                validation_data=validation_generator,
                                validation_steps=validation_generator.samples // validation_generator.batch_size
                                )


Epoch 1/20
58/58 [==============================] - 156s 3s/step - loss: 1.6096 - accuracy: 0.1968 - val_loss: 1.6096 - val_accuracy: 0.1953
Epoch 2/20
58/58 [==============================] - 151s 3s/step - loss: 1.6088 - accuracy: 0.2065 - val_loss: 1.6056 - val_accuracy: 0.2352
Epoch 3/20
58/58 [==============================] - 152s 3s/step - loss: 1.5759 - accuracy: 0.2664 - val_loss: 1.5142 - val_accuracy: 0.2965
Epoch 4/20
58/58 [==============================] - 152s 3s/step - loss: 1.4663 - accuracy: 0.3300 - val_loss: 1.4084 - val_accuracy: 0.3495
Epoch 5/20
58/58 [==============================] - 151s 3s/step - loss: 1.3685 - accuracy: 0.3762 - val_loss: 1.3329 - val_accuracy: 0.4128
Epoch 6/20
58/58 [==============================] - 152s 3s/step - loss: 1.2764 - accuracy: 0.4273 - val_loss: 1.1547 - val_accuracy: 0.5107
Epoch 7/20
58/58 [==============================] - 152s 3s/step - loss: 1.1552 - accuracy: 0.4985 - val_loss: 1.0126 - val_accuracy: 0.5905
Epoch 8/20
58

# Save the CNN model

In [ ]:
B1_CNN = 'B1_1_CNN.h5'

cnn_model.save(B1_CNN)

# Evaludate on the Test Dataset

In [ ]:
# extracting the test dataset
cartoon_test_label = pd.read_csv('./Datasets/cartoon_set_test/labels.csv', sep = '\t')
face_test_label = cartoon_test_label.drop(columns = [cartoon_test_label.columns[0]]).drop(columns = [cartoon_test_label.columns[1]])
face_test_label['face_shape'] = face_test_label['face_shape'].apply(str)
print(face_test_label)

     face_shape file_name
0             1     0.png
1             4     1.png
2             2     2.png
3             2     3.png
4             3     4.png
...         ...       ...
2495          4  2495.png
2496          4  2496.png
2497          4  2497.png
2498          4  2498.png
2499          4  2499.png

[2500 rows x 2 columns]


In [ ]:
# image path for new test images
cartoon_test_img = ('./Datasets/cartoon_set_test/img')

In [ ]:
test_generator = data_argumantation.flow_from_dataframe(
        dataframe = face_test_label, 
        directory = cartoon_test_img,
        x_col = "file_name", 
        y_col = "face_shape",
        batch_size=1,
        class_mode='categorical', 
        target_size=(128,128),
        shuffle=False)
test_steps = test_generator.samples
print(test_steps)

Found 2500 validated image filenames belonging to 5 classes.
2500


In [ ]:
# testing model on new dataset
B1_model = load_model(B1_CNN)

In [ ]:
# printing training loss and accuracy
train_score = B1_model.evaluate(train_generator)

print('Train loss: '+ str(train_score[0]))
print('Train Accuracy: '+ str(train_score[1]))

59/59 [==============================] - 94s 2s/step - loss: 0.2760 - accuracy: 0.9076
Train loss: 0.27603575587272644
Train Accuracy: 0.9075999855995178


In [ ]:
# printing validation loss and accuracy
valid_score = B1_model.evaluate(validation_generator)
                                          
print('Validation loss: '+ str(valid_score[0]))
print('Validation Accuracy: '+ str(valid_score[1]))

20/20 [==============================] - 32s 2s/step - loss: 0.2631 - accuracy: 0.9172
Validation loss: 0.26313766837120056
Validation Accuracy: 0.9172000288963318


In [ ]:
test_generator.reset()
B1_pred = B1_model.predict(test_generator)

# determine the maximum activation value for each sample
B1_pred_cnn = np.argmax(B1_pred, axis=1)

# label each predicted value to correct face shape
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in B1_pred_cnn]

2500/2500 [==============================] - 496s 198ms/step


In [ ]:
accuracy_cnn = accuracy_score(test_generator.classes, B1_pred_cnn)

In [ ]:
print(classification_report(test_generator.classes, B1_pred_cnn ))
print("Accuracy of CNN model: {:.2f}%".format(accuracy_cnn*100))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       500
           1       0.81      0.86      0.84       500
           2       0.94      0.95      0.94       500
           3       0.88      0.88      0.88       500
           4       0.96      0.95      0.95       500

    accuracy                           0.90      2500
   macro avg       0.90      0.90      0.90      2500
weighted avg       0.90      0.90      0.90      2500

Accuracy of CNN model: 90.32%
